In [1]:
import os
os.chdir('/home/reffert/DeepAR_InfluenzaForecast')
from PythonFiles.model import model, preprocessing, split_forecasts_by_week, plot_coverage, print_forecasts_by_week, forecast_by_week, train_test_split, update_deepAR_parameters
from PythonFiles.Configuration import Configuration
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from gluonts.mx import Trainer, DeepAREstimator
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.rolling_dataset import generate_rolling_dataset,StepStrategy
from ray import tune
config = Configuration()
influenza_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/influenza.csv", sep=',')
population_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/PopulationVector.csv", sep=',')
neighbourhood_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/AdjacentMatrix.csv", sep=',', index_col=0)

In [2]:
config.train_start_time = datetime(1999,1,1,0,0,0)#datetime(2010,1,1,0,0,0)
config.train_end_time = datetime(2016,9,30,23,0,0)
config.test_end_time = datetime(2018,9,30,23,0,0)
overall_evaluation_df = pd.DataFrame()

data_splits_dict = {}
output_dict = {}

locations = list(influenza_df.location.unique())
#Process the df into a uniformly spaced df
df = influenza_df.loc[influenza_df.location.isin(locations), ['value', 'location', 'date','week']]
df = preprocessing(config, df, check_count=False, output_type="corrected_df")
for location in locations:
    df.loc[df.location == location, "population"] = int(population_df.loc[population_df.Location == location, "2011"].values[0])
    df.loc[df.location == location, locations] = neighbourhood_df.loc[neighbourhood_df.index==location,locations].values[0]

In [3]:
# Data split with no additional features and training start in 2010
#data_splits_dict["without_features_2001"] = list(train_test_split(config, df, False))
data_splits_dict["with_features_2001"] = list(train_test_split(config, df, True))

# Change the beginning of the training period
#config.train_start_time = datetime(2010,1,1,0,0,0)
#data_splits_dict["without_features_2010"] = list(train_test_split(config, df, False))
#data_splits_dict["with_features_2010"] = list(train_test_split(config, df, True))

In [4]:
# define a function that should be optimized 
def train_deepar(config, hp_search_space):
    tuned_model = update_deepAR_parameters(config, hp_search_space)
    model_dict = {"DeepAR":tuned_model}#, "FFNN":config.feedforwardestimator}
    forecasts_dict, evaluator_df_dict = forecast_by_week(config, data_splits_dict["with_features_2001"][0], data_splits_dict["with_features_2001"][1], locations, model_dict)
    evaluation_df = evaluator_df_dict["DeepAR"].copy()
    evaluation_df = evaluation_df.loc[evaluation_df.item_id.isin([item_id for item_id in evaluation_df.item_id if "aggregate" in item_id]),]        
    return evaluation_df.mean_WIS.mean()

In [5]:
hp_search_space = {
    "num_layers": 1,
    "num_cells": 2,
    "learning_rate": 1e-1,
    #"batch_size":2,
    "epochs": 2,
}
evaluation_df = train_deepar(config, hp_search_space)
print(3)

1


100%|██████████| 50/50 [00:03<00:00, 14.68it/s, epoch=2/2, avg_epoch_loss=0.88]


KeyboardInterrupt: 

In [ ]:
hp_search_space = {
    "num_layers": tune.choice([1, 2, 3]),
    "num_cells": tune.choice([32, 64, 128]),
    "learning_rate": tune.loguniform(1e-5, 1e-1),
    "dropout_rate": tune.uniform(0.1, 0.3),
    "batch_size": tune.choice([32, 64, 128]),
    "epochs": tune.choice([10, 20, 30])
}

print(evaluation_df)
print(evaluation_df.mean_WIS.mean())